In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

# Constants
IMAGE_SIZE = (128, 128)  # Resize images to 128x128
BATCH_SIZE = 32

# Load labels from the CSV file
def load_labels_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    filenames = df['image_name'].values
    classes = df['tags'].values
    return filenames, classes

# Load images based on filenames from CSV
def load_images_from_folder(folder, filenames):
    images = []
    i =  0
    for filename in filenames:
        # if i > 1000:
        #     break
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, IMAGE_SIZE)
                images.append(img)
                
    return np.array(images)




In [2]:
from sklearn.preprocessing import LabelEncoder


# Load data
filenames, y = load_labels_from_csv('/Users/angel/OneDrive/Desktop/CS 4100/Satellite_Image_Classifier/data/train_v2.csv')
filenames = [name + '.jpg' for name in filenames]

image_folder = '/Users/angel/OneDrive/Desktop/CS 4100/train-jpg/train-jpg/'
X = []

X = load_images_from_folder(image_folder, filenames)
print(np.shape(filenames))
print(filenames[0:10])
# Normalize the images
X = X.astype('float32') / 255.0

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


print('X: ', len(X), '\nY: ', len(y))

(40479,)
['train_0.jpg', 'train_1.jpg', 'train_2.jpg', 'train_3.jpg', 'train_4.jpg', 'train_5.jpg', 'train_6.jpg', 'train_7.jpg', 'train_8.jpg', 'train_9.jpg']
X:  40479 
Y:  40479


In [5]:
num_classes = len(np.unique(y_encoded))
print(num_classes)

449


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y)), activation='softmax')  # Use the number of unique classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15, batch_size=BATCH_SIZE, validation_split=0.2)


predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

accuracy = accuracy_score(predicted_classes, y_test)

#test_loss, test_acc = model.evaluate(X_test, y_test)
#print(f"Test accuracy: {test_acc}")

# Example of printing a prediction
for i in range(5):  # Print first 5 predictions
    print(f"Predicted class for test image {i}: {predicted_classes[i]} (Actual class: {y_test[i]})")